# Importing the libraries we need

First we need to import everything we will need for this project.

In [1]:
# Import necessary libraries
%matplotlib inline

import folium
import geocoder
import requests
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from sklearn.cluster import KMeans

# Importing the dataset

Using read_html function as a way to scrape wikipedia table however it returns a list of tables it found hence returning only the first table that it seems to be our needs.

In [2]:
# Read all the tables in that page
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
# Getting only the first table as our dataframe
df = tables[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Cleaning the dataset

Time to clean up our dataset (taking care of missing values) to have only the available boroughs plus assigning neighborhoods to their boroughs in case they were not assigned hence we have no missing values.

In [3]:
# Creating a new DataFrame out of the old one to have only available boroughs
df = pd.DataFrame(df[df["Borough"] != "Not assigned"])
# Replacing neighborhoods "not assigned" to NaN
df["Neighborhood"].replace("Not assigned", np.nan, inplace=True)
# Fill NaN with their corresponding boroughs
df["Neighborhood"].fillna(df["Borough"], inplace=True)
# Reset index and drop the old index column
df.reset_index(inplace=True, drop=True)
# Show first five rows
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# How many rows we got?

Displaying the number of rows we deduced out of the dataset we scraped and as we can see, 103 rows are extracted.

In [4]:
df.shape

(103, 3)

# Getting coordinates of each neighborhood

Next, we have to get coordinates of each neighborhood and add them to the dataset. First, we'll read the csv file coursera provided to us (since the API is really unreliable when I tested it out).

In [5]:
# Read coordinates CSV file
df_coords = pd.read_csv("Geospatial_Coordinates.csv")
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Next, merge both coordinates dataframe and neighborhood dataframe.

In [6]:
# Merge df and df_coords to one dataframe
new_df = pd.merge(df, df_coords, on="Postal Code")
new_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Exploring Neighborhoods in Toronto

Time to explore our dataset we generated. There are some steps first before getting started to explore the neighborhoods.

### 1. Define Foursquare Credentials.

In [7]:
CLIENT_ID = 'CH4WIJGKRIAWASFEHPBMMZ0KTMHNK5452UANX2XDD3ELKNCU'
CLIENT_SECRET = 'IS04REXKHWFXKY25DIEQV24SMO4YYYIBVVRSD3HAUMN3QCUE'
VERSION = '20180605'

### 2. Define getNearbyVenues function (borrowed from the lab).

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
        # make the GET request
        while True:
            try:
                results = requests.get(url).json()["response"]['groups'][0]['items']
                break
            except Exception as e:
                print(e)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return nearby_venues

### 3. Define toronto boroughs

In [9]:
toronto_df = new_df[new_df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


### 4. Define the venues variable.

In [10]:
toronto_venues = getNearbyVenues(names=new_df['Neighborhood'], latitudes=new_df['Latitude'], longitudes=new_df['Longitude'])
toronto_venues.head()

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Brookbanks Pool,43.751389,-79.332184,Pool
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


# Analyzing the exploratory data

First we need to explore how many venues found and how many unique categories there are.

In [11]:
print(f"{toronto_venues.shape[0]} venues found.")
print(f'{len(toronto_venues["Venue Category"].unique())} unique categories.')

2125 venues found.
267 unique categories.


Next we need to onehot each neighborhood.

In [12]:
# One-hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# Add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
# Check first five rows
toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Then, group records and calculate the frequency of each venue category.

In [13]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038462,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Sorting the top 10 common venues

Sorting most common venues is the next part to cluster them into 2 clusters. That would take two additional steps.

### 1. Define a function that sorts our venues in descending order (borrowed from the lab).

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[:num_top_venues]

### 2. Create a new dataframe with the top 10 venues appended.

In [15]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# Create columns according to number of top venues
columns = ['Neighborhood']
for i in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Most Common Venue'.format(i+1))
# Create a new dataframe with the top 10 common venues appended
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']
# Loop through the rows
for i in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[i, 1:] = return_most_common_venues(toronto_grouped.iloc[i, :], num_top_venues)
# Check first five rows
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Clothing Store,Breakfast Spot,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
1,"Alderwood, Long Branch",Pizza Place,Gym,Coffee Shop,Skating Rink,Sandwich Place,Pub,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Frozen Yogurt Shop,Supermarket,Sushi Restaurant,Middle Eastern Restaurant,Shopping Mall,Mobile Phone Shop,Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Restaurant,Juice Bar,Pharmacy,Indian Restaurant,Pub,Café,Sushi Restaurant


# Clustering the neighborhoods

Now when we are done with cleaning up the dataset plus sorting them, it's time to cluster! First we will use k-Means to cluster them into 2 clusters.

In [16]:
# Defnining the number of clusters
n_clusters = 2
# Drop neighborhood column
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# Run k-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(toronto_grouped_clustering)
# Check the first 5 labels
kmeans.labels_[:5]

array([1, 1, 1, 1, 1])

Next we add the labels that k-Means extracted for us to the original dataframe.

In [17]:
# Insert the cluster labels
neighborhoods_venues_sorted.insert(0, 'Label', kmeans.labels_)
toronto_merged = toronto_df
# Merge the original dataframe with the labels
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
# Check the first five rows
toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Ice Cream Shop,Performing Arts Venue,Shoe Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Gym,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop,General Entertainment,Fried Chicken Joint
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Italian Restaurant,Hotel,Japanese Restaurant,Fast Food Restaurant,Electronics Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Restaurant,Cosmetics Shop,Clothing Store,Cocktail Bar,American Restaurant,Moroccan Restaurant,Breakfast Spot,Gym
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Pub,Health Food Store,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Japanese Restaurant
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Japanese Restaurant,Bar,Burger Joint,Thai Restaurant,Salad Place
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Nightclub,Baby Store,Athletics & Sports,Candy Store,Italian Restaurant,Diner,Coffee Shop
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,1,Coffee Shop,Café,Hotel,Restaurant,Gym,Thai Restaurant,Deli / Bodega,Salad Place,Pizza Place,Sushi Restaurant
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Bakery,Pharmacy,Grocery Store,Middle Eastern Restaurant,Park,Brewery,Pet Store,Café,Supermarket,Music Venue


Lastly we visualize those cluters via a map using folium.

In [18]:
# Creating the toronto map
toronto_map = folium.Map(location=[toronto_merged["Latitude"][0], toronto_merged['Longitude'][0]], zoom_start=11)
# Choose different colors for each cluster to differentiate
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Adding markers to each neighborhood
markers_colors = []
for lat, lon, neighborhood, cluster_label in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Label']):
    label = folium.Popup(f"{neighborhood} Cluster {cluster_label}", parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster_label-1],
        fill=True,
        fill_color=rainbow[cluster_label-1],
        fill_opacity=0.7).add_to(toronto_map)
# Showing the map
toronto_map

In case you don't see the map, here is the link: https://imgur.com/SXVvZqD

# Analyzing the clusters

Next we need to know why they were clustered that way hence we'll check each cluster's venues to see if there is any relation.

### First Cluster

In [26]:
toronto_merged.loc[toronto_merged['Label'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Bus Line,Swim School,Diner,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Discount Store,Ethiopian Restaurant
21,"Forest Hill North & West, Forest Hill Road Park",Trail,Park,Sushi Restaurant,Jewelry Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
29,"Moore Park, Summerhill East",Park,Trail,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Curling Ice
33,Rosedale,Park,Playground,Trail,Donut Shop,Doner Restaurant,Drugstore,Dog Run,Distribution Center,Discount Store,Cuban Restaurant


From my own prespective I see this cluster mainly consists of popular trails and parks.

### Second Cluster

In [27]:
toronto_merged.loc[toronto_merged['Label'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Ice Cream Shop,Performing Arts Venue,Shoe Store
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Diner,Gym,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop,General Entertainment,Fried Chicken Joint
2,"Garden District, Ryerson",Clothing Store,Coffee Shop,Cosmetics Shop,Bubble Tea Shop,Café,Italian Restaurant,Hotel,Japanese Restaurant,Fast Food Restaurant,Electronics Store
3,St. James Town,Café,Coffee Shop,Restaurant,Cosmetics Shop,Clothing Store,Cocktail Bar,American Restaurant,Moroccan Restaurant,Breakfast Spot,Gym
4,The Beaches,Trail,Pub,Health Food Store,Discount Store,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
5,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Café,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Japanese Restaurant
6,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Bubble Tea Shop,Japanese Restaurant,Bar,Burger Joint,Thai Restaurant,Salad Place
7,Christie,Grocery Store,Café,Park,Nightclub,Baby Store,Athletics & Sports,Candy Store,Italian Restaurant,Diner,Coffee Shop
8,"Richmond, Adelaide, King",Coffee Shop,Café,Hotel,Restaurant,Gym,Thai Restaurant,Deli / Bodega,Salad Place,Pizza Place,Sushi Restaurant
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Grocery Store,Middle Eastern Restaurant,Park,Brewery,Pet Store,Café,Supermarket,Music Venue


From my own prespective I see this cluster mainly consists of popular various services.

### Thanks for reviewing my report!

This notebook was created by [devkarim](https://github.com/devkarim). I wish you found this report interesting and helpful!